In [1]:
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import importlib
from scipy.io import FortranFile

import time 
import cftime 

import RegridField as RgF

# import modules in other directories
sys.path.append('../Utils/')
import GridUtils as GrU
import MakePressures as MkP
import humiditycalcs as hum
import MyConstants as Con

sys.path.append('../Regridder/')
import Initialize as Prep
import ReadInSrc as RdSrc
import WriteDST as WrDst
import GenRegrid as GR
import latlon_w_scrip as LL
import var_A_x_B as AxB


importlib.reload( RgF )
importlib.reload( AxB )
importlib.reload( LL )
importlib.reload( GR )
importlib.reload( Prep )
importlib.reload( RdSrc )
importlib.reload( WrDst )


 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
Using Flexible parallel/serial VertRegrid 
 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
 a path added in GenRegrid /glade/work/juliob/PyRegridding/Regridder/../Utils 
Using Flexible parallel/serial VertRegrid 
 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
 a path added in var_A_x_B /glade/work/juliob/PyRegridding/Regridder/../Utils 
Using Flexible parallel/serial VertRegrid 
 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
 a path added in var_A_x_B /glade/work/juliob/PyRegridding/Regridder/../Utils 
 a path added i

<module 'WriteDST' from '/glade/work/juliob/PyRegridding/Drivers/../Regridder/WriteDST.py'>

In [2]:
##########################

restF = '/glade/derecho/scratch/juliob/archive/c64_gwR2_ne30pg3_FMTHIST_topfix_oldCTL_qbotune_x03/rest/1997-01-01-00000/c64_gwR2_ne30pg3_FMTHIST_topfix_oldCTL_qbotune_x03.cam.i.1997-01-01-00000.nc'


#restF= '/glade/campaign/cgd/amp/aherring/restart/inic/ne120/ERA5_x_ne120np4_L58_rgC2_WO.aero.2000-01-01-00000.nc'

#restF ='/glade/derecho/scratch/juliob/archive/c64_gwR2_ne30pg3_FMTHIST_topfix_rdgres_x02/rest/1987-01-01-00000/c64_gwR2_ne30pg3_FMTHIST_topfix_rdgres_x02.cam.i.1987-01-01-00000.nc'
RestX = xr.open_dataset(restF) 

#eraF  ='/glade/campaign/cgd/amp/juliob/ERA5/ne120np4/L32/ERA5_x_ne120np4_L32_rgC2_WO.2000-01-01-00000.nc'

eraF  ='/glade/campaign/cgd/amp/juliob/ERA5/ne30np4/L120/ERA5_x_ne30np4_L120_rgC2_WO.1985-01-01-00000.nc'
eraX  = xr.open_dataset(eraF) 


RestXvars = list(RestX.variables) 
eraXvars  = list(eraX.variables) 

so4_a1 = RestX.so4_a1.values
so2    = RestX.SO2.values
ps     = RestX.PS.values


print( np.shape(so4_a1) )

(1, 93, 48602)


In [3]:
print('lev' in RestX.SO2.dims)

True


In [4]:
import RegridField as RgF
import copy
importlib.reload( RgF )

eraX_xtnd = copy.deepcopy( eraX )
ps     = eraX.PS.values

for fld in RestXvars:
    Q1 = (('lev' in RestX[fld].dims) or ('ilev' in RestX[fld].dims) )
    Q2 = (fld not in eraXvars )
    #print( fld, Q1, Q2 )
    if ( (Q1==True ) and ( Q2==True) ):
        #print( fld, RestX[fld].dims )
        aa = RestX[fld].values
        aa_z = RgF.Vert(DstVgrid='L120', DstTZHkey='tzc', SrcVgrid='L93', xfld_Src=aa, ps_Src=ps )
        print( fld, RestX[fld].attrs , np.shape(aa_z) )

        
        Dar = xr.DataArray( data=aa_z, 
                            dims=('time','lev','ncol',),
                            attrs=RestX[fld].attrs ,)
        
        eraX_xtnd[fld] = Dar
                


Using Flexible parallel/serial VertRegrid 
 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
Flexible VertRegrid using reshaped ARRAYS 
nworkers available  1
Using serial code which is faster for 1 worker
Pll'zd Vertical int 2.9811 seconds
CFC11 {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'dry', 'long_name': 'CFC11', 'cell_methods': 'time: point'} (1, 120, 48602)
Flexible VertRegrid using reshaped ARRAYS 
nworkers available  1
Using serial code which is faster for 1 worker
Pll'zd Vertical int 2.9117 seconds
CFC12 {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'dry', 'long_name': 'CFC12', 'cell_methods': 'time: point'} (1, 120, 48602)
Flexible VertRegrid using reshaped ARRAYS 
nworkers available  1
Using serial code which is faster for 1 worker
Pll'zd Vertical int 3.7310 seconds
CH4 {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'dry', 'long_name': 'CH4', 'cell_methods': 'time: point'} (1, 120,

In [5]:
# Make destination file name by replacing '_WO.'

#eraF  ='/glade/campaign/cgd/amp/juliob/ERA5/ne30np4/L32/ERA5_x_ne30np4_L32_rgC2_WO.2000-01-01-00000.nc'

eraF_xtnd = eraF.replace('_WO.', '_WO_aero.' )
print(eraF_xtnd)
eraX_xtnd.to_netcdf(  eraF_xtnd )

/glade/campaign/cgd/amp/juliob/ERA5/ne30np4/L120/ERA5_x_ne30np4_L120_rgC2_WO_aero.1985-01-01-00000.nc


In [6]:
eraX_xtnd

<xarray.Dataset>
Dimensions:    (ncol: 48602, lev: 120, ilev: 121, time: 1)
Coordinates:
    lon        (ncol) float64 ...
    lat        (ncol) float64 ...
  * lev        (lev) float64 0.007676 0.0111 0.01583 ... 983.2 991.2 997.5
  * ilev       (ilev) float64 0.006247 0.009106 0.01309 ... 987.4 995.1 1e+03
  * time       (time) int32 0
Dimensions without coordinates: ncol
Data variables: (12/55)
    TimeStamp  datetime64[ns] ...
    P_00       float64 ...
    hyai       (ilev) float64 ...
    hybi       (ilev) float64 ...
    hyam       (lev) float64 ...
    hybm       (lev) float64 ...
    ...         ...
    pom_a4     (time, lev, ncol) float64 1.817e-19 2.373e-19 ... 8.945e-11
    so4_a1     (time, lev, ncol) float64 -8.587e-23 -7.779e-23 ... 3.201e-10
    so4_a2     (time, lev, ncol) float64 5.639e-15 7.317e-15 ... 2.045e-12
    so4_a3     (time, lev, ncol) float64 -4.196e-26 -4.479e-26 ... 4.314e-13
    soa_a1     (time, lev, ncol) float64 1.459e-30 1.928e-30 ... 1.156e-10
    soa_a2     (time, lev, ncol) float64 6.645e-25 7.488e-25 ... 6.54e-13